# 05_funcGLM2

这是功能像的1st GLM。

In [ ]:
import os
import scipy.io as scio
from RZutilpy.system import Path, unix_wrapper, gettimestr, makedirs
from RZutilpy.rzio import matchfiles

# Step 1. 

Load group information

In [ ]:
Group = scio.loadmat('/home/data/yufengspace/23BPhappy/temp/clidata.mat')
Group = Group['clidata']

GroupMDD = []
GroupBD = []
#del folders[0:12] 

Param1 = 'option_CRandEV'
Param2 = 'gamble_RPE'
for i in range(0,len(Group)):
    subj = Group[i][0][0]
    if Group[i][10][0][0] == 0:
        GroupMDD.append(subj)
    elif Group[i][10][0][0] == 1:
        GroupBD.append(subj)

    #statsfiles = matchfiles(f'{GLM_dir}/*{Param1}_{Param2}_stats+tlrc.BRIK')
    #!cp {statsfiles}  /home/yufengxia/Data/23BPhappy/temp
    #print(statsfiles)

# Step 2. 

Calculate Group mask

In [ ]:

folders = matchfiles(f'/home/data/yufengspace/23BPhappy/*_BPhappy_*')
for folderofsubj in folders:
    subj = folderofsubj[-4:]
    func_dir = Path(f'{folderofsubj}/funcpp')
    func_dir = Path(func_dir)
    if subj in GroupMDD: 
        ! mv {func_dir}/mask_epi_anat.{subj}.nii.gz /home/data/yufengspace/23BPhappy/temp # move all masks of individuals to temp dir
    if subj in GroupBD:
        ! mv {func_dir}/mask_epi_anat.{subj}.nii.gz /home/data/yufengspace/23BPhappy/temp

#create group mask
%cd /home/data/yufengspace/23BPhappy/temp/
! 3dmask_tool -input mask_epi_anat.* \
        -prefix Group_mask_inter -inter # calculate an intersection of the masks, or you can choose calculate an union mask 

# Step 3. 

Generating the Results

In [ ]:
%cd /home/data/yufengspace/23BPhappy/results/
! tcsh /home/yufengxia/Code/yfXRepo/CCNN-23BPhappy/fMRI_Proc/scripts/cmd.funcGLM2.23BPhappy 
! rm *minmax* *cmd* 

In [ ]:
Param1 = 'option_CRandEV'
Param2 = 'gamble_RPE'
folders = matchfiles(f'/home/data/yufengspace/23BPhappy/*_BPhappy_*')

setA = '-setA' 
setB = '-setB'
for folderofsubj in folders:
    subj = folderofsubj[-4:]
    GLM_dir = Path(f'{folderofsubj}/funcGLM1')
    GLM_dir = Path(GLM_dir)
    files =  f'{GLM_dir}/{subj}_{Param1}_{Param2}_stats+tlrc[12]'
    if subj in GroupMDD:     
        setA += ' '  + files 
    if subj in GroupBD:
        setA += ' '  + files 
%cd /home/data/yufengspace/23BPhappy/results/
print(setA)
! 3dttest++ -Clustsim -mask Group_mask_inter+tlrc \
    -prefix GLM2_ALL_{Param2}_loop {setA} -labelA ALL 


setA = '-setA' 
setB = '-setB'
for folderofsubj in folders:
    subj = folderofsubj[-4:]    
    GLM_dir = Path(f'{folderofsubj}/funcGLM1')
    GLM_dir = Path(GLM_dir)
    files = f'{GLM_dir}/{subj}_{Param1}_{Param2}_stats+tlrc[12]'
    if subj in GroupMDD:
        setA += ' ' + files 
    if subj in GroupBD:
        setB += ' ' + files 


print(setA)
print(setB)
%cd /home/data/yufengspace/23BPhappy/results/
! 3dttest++ -Clustsim -mask Group_mask_inter+tlrc.HEAD\
    -prefix GLM2_CT_{Param2}.dset {setA} {setB} -labelA MDD -labelB BD

In [ ]:
Param1 = 'option_CRandEV'
Param2 = 'gamble_RPE'
seed = 'rvs'
folders = matchfiles(f'/home/data/yufengspace/23BPhappy/*_BPhappy_*')

setA = '-setA' 
for folderofsubj in folders:
    subj = folderofsubj[-4:]
    ppi_dir = Path(f'{folderofsubj}/funcppi')
    ppi_dir = Path(ppi_dir)
    files = f'{ppi_dir}/PPI.{subj}.{seed}.stats.+tlrc[16]' #注意这里比上面GLM1文件夹里的stats文件命名多了一个‘.’
    if subj in GroupMDD:     
        setA += ' '  + files 
    if subj in GroupBD:
        setA += ' '  + files 

print(setA)
%cd /home/data/yufengspace/23BPhappy/results/
! 3dttest++ -Clustsim -mask Group_mask_uni+tlrc.HEAD\
    -prefix GLM2_PPIALL_{seed}_{Param2}.dset {setA} -labelA ALL 


setA = '-setA' 
setB = '-setB'
for folderofsubj in folders:
    subj = folderofsubj[-4:]
    ppi_dir = Path(f'{folderofsubj}/funcppi')
    ppi_dir = Path(ppi_dir)
    files = f'{ppi_dir}/PPI.{subj}.{seed}.stats.+tlrc[16]'
    if subj in GroupMDD:
        setA += ' '  + files 
    if subj in GroupBD:
        setB += ' '  + files

print(setA)
print(setB)
%cd /home/data/yufengspace/23BPhappy/results/
! 3dttest++ -Clustsim -mask Group_mask_uni+tlrc.HEAD\
    -prefix GLM2_PPICT_{seed}_{Param2}.dset {setA} {setB} -labelA MDD -labelB BD

In [ ]:
#可视化结果
Param1 = 'option_CRandEV'
Param2 = 'gamble_RPE'
AFNI_HOME = os.getenv('AFNIDIR')
MNIT1 = AFNI_HOME + '/MNI152_2009_template_SSW.nii.gz'
%cd /home/data/yufengspace/23BPhappy/results/
! afni -dset GLM2_CT_{Param2}+tlrc.BRIK {MNIT1}